In [1]:
# pip install transformers datasets

     ---------------------------------------- 0.0/44.1 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.1 kB 640.0 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/44.1 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 44.1/44.1 kB 360.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.6 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 764.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/69.9 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/69.9 kB ? eta -:--:--
     ---------------- --------------------- 30.7/69.9 kB 640.0 kB/s eta 0:00:01
     --------------------------------- ---- 61.4/69.9 kB 544.7 kB/s eta 0:00:01


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

Using the latest cached version of the dataset since cnn_dailymail couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '3.0.0' at C:\Users\vishn\.cache\huggingface\datasets\cnn_dailymail\3.0.0\0.0.0\96df5e686bee6baa90b8bee7c28b81fa3fa6223d (last modified on Mon Dec  9 12:05:11 2024).


In [2]:
# import matplotlib.pyplot as plt
# import seaborn as sns


# # Compute lengths
# article_lengths = [len(article.split()) for article in dataset['train']['article']]
# summary_lengths = [len(summary.split()) for summary in dataset['train']['highlights']]

# # Plotting Article Lengths
# plt.figure(figsize=(14, 6))
# sns.histplot(article_lengths, bins=50, color='blue')
# plt.title('Distribution of Article Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

# # Plotting Summary Lengths
# plt.figure(figsize=(14, 6))
# sns.histplot(summary_lengths, bins=100, color='red')
# plt.title('Distribution of Summary Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

In [3]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Join all summaries
# text = " ".join(summary for summary in dataset['train']['highlights'])

# # Optionally, you can clean the text (remove stopwords, punctuation, etc.)
# # For example:
# # text = clean_text_function(text)

# # Generate word cloud
# wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# # Plot the word cloud
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')  # Disable axis
# plt.show()


In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('lucadiliello/bart-small')
model = BartForConditionalGeneration.from_pretrained('lucadiliello/bart-small', ignore_mismatched_sizes=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [5]:
def preprocess_function(examples):
    # Tokenize the articles and summaries (highlights)
    inputs = tokenizer(examples['article'], padding='max_length', truncation=True, max_length=512)
    labels = tokenizer(examples['highlights'], padding='max_length', truncation=True, max_length=150)

    # Ensure that labels are correctly aligned with the input format
    inputs['labels'] = labels['input_ids']
    return inputs


# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [6]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")

Using device: cuda


In [7]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(  
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',
    report_to="none",
    save_steps=500,
    save_total_limit=3, 
    remove_unused_columns=False,
    load_best_model_at_end=True,
    eval_strategy="steps",
    eval_steps=500,
)

# Initialize Trainer
trainer = Trainer(
    model=model.to('cuda:0'),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

In [8]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
c:\Users\vishn\OneDrive\Desktop\GUI\creategui\Lib\site-packages\transformers\trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have fu

  0%|          | 0/215337 [00:00<?, ?it/s]

c:\Users\vishn\OneDrive\Desktop\GUI\creategui\Lib\site-packages\transformers\trainer.py:3033: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load

{'loss': 0.927, 'grad_norm': 2.847473382949829, 'learning_rate': 3.2203484502203998e-06, 'epoch': 2.81}


  0%|          | 0/3342 [00:00<?, ?it/s]

...

  0%|          | 0/3342 [00:00<?, ?it/s]

{'eval_loss': 0.9916338920593262, 'eval_runtime': 485.3872, 'eval_samples_per_second': 27.541, 'eval_steps_per_second': 6.885, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


{'train_runtime': 18466.5083, 'train_samples_per_second': 46.643, 'train_steps_per_second': 11.661, 'train_loss': 0.062208103107353245, 'epoch': 3.0}


TrainOutput(global_step=215337, training_loss=0.062208103107353245, metrics={'train_runtime': 18466.5083, 'train_samples_per_second': 46.643, 'train_steps_per_second': 11.661, 'total_flos': 1.1679735407129395e+17, 'train_loss': 0.062208103107353245, 'epoch': 3.0})

In [9]:
model.save_pretrained('Model')
tokenizer.save_pretrained('Model')

('Model\\tokenizer_config.json',
 'Model\\special_tokens_map.json',
 'Model\\vocab.json',
 'Model\\merges.txt',
 'Model\\added_tokens.json')

In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('Model')
tokenizer = BartTokenizer.from_pretrained('Model')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [13]:
# Example input article
article = """
The rise of artificial intelligence (AI) has dramatically changed many industries over the past few years. AI, a branch of computer science that deals with the creation of intelligent machines capable of performing tasks that normally require human intelligence, is becoming a ubiquitous part of modern life. From healthcare and transportation to entertainment and manufacturing, AI has made a profound impact on the way we live and work.

In the healthcare sector, AI has proven to be an invaluable tool for doctors and medical professionals. It aids in diagnosing diseases, predicting patient outcomes, and personalizing treatments. Machine learning algorithms can analyze vast amounts of medical data much faster than a human, enabling healthcare providers to make more accurate and timely decisions. AI is also being used in drug discovery, helping researchers identify potential compounds that could lead to life-saving medications. In fact, AI has accelerated the pace of pharmaceutical research, leading to the faster development of vaccines and other critical drugs.

In transportation, AI is a key driver behind the development of autonomous vehicles. Self-driving cars use a combination of sensors, cameras, and AI algorithms to navigate and make decisions without human intervention. The promise of fully autonomous vehicles is revolutionizing the way we think about transportation, offering potential benefits like reduced traffic accidents, improved fuel efficiency, and less congestion in urban areas. AI also plays a critical role in logistics and supply chain management, optimizing routes for delivery trucks, predicting demand, and ensuring that goods are delivered on time.

The entertainment industry has also seen a surge in AI-powered technologies. Streaming platforms like Netflix and Spotify rely on AI algorithms to recommend content based on user preferences. These algorithms analyze vast amounts of data to provide personalized suggestions, improving user experience and engagement. AI is also making waves in the world of video games, where it is used to create more realistic and immersive environments. AI-driven characters can learn from player behavior, adapting and responding in ways that make the game more dynamic and challenging.

Manufacturing industries are utilizing AI for predictive maintenance and quality control. By analyzing sensor data from machines, AI algorithms can predict when equipment is likely to fail, reducing downtime and maintenance costs. AI also helps in optimizing production processes, ensuring that products are made more efficiently and with fewer defects. In this way, AI contributes to the overall competitiveness of businesses, allowing them to stay ahead in an increasingly globalized market.

Despite the many benefits, the rise of AI also brings challenges. One major concern is the potential for job displacement. As AI technologies become more advanced, many tasks that were once performed by humans are now being automated. This has led to concerns about unemployment, especially in sectors like manufacturing, transportation, and customer service. There are also ethical questions surrounding AI, such as the use of AI in surveillance and its potential impact on privacy.

Furthermore, there is the question of AI governance. As AI continues to evolve, it is crucial to establish frameworks and regulations to ensure that AI is developed and used responsibly. Policymakers and industry leaders are working to create standards that can prevent misuse and ensure that AI benefits society as a whole. This involves balancing innovation with ethics, and ensuring that AI systems are transparent, accountable, and fair.

In conclusion, AI has the potential to revolutionize the world, bringing about positive changes in fields such as healthcare, transportation, entertainment, and manufacturing. However, it also presents challenges that must be addressed to ensure that its benefits are realized in a way that is both ethical and equitable. As we move forward, it is important to continue exploring the possibilities of AI while also considering the societal implications it brings. The future of AI holds great promise, but it requires careful thought, regulation, and responsible development.
"""
# Tokenize the input article
inputs = tokenizer(article, return_tensors="pt", padding='max_length', truncation=True, max_length=512)


In [14]:
# Generate summary using the fine-tuned model
with torch.no_grad():  # Avoid gradient calculation during inference
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)

# Decode the summary from the token IDs
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated Summary: ", summary)

Generated Summary:  The rise of artificial intelligence (AI) has dramatically changed many industries over the past few years .
AI is a branch of computer science that deals with the creation of intelligent machines capable of performing tasks that normally require human intelligence .
From healthcare and transportation to entertainment and manufacturing, AI has made a profound impact on the way we live and work .
